# PTUSE observation report

In [ ]:
print 'Report Started:'
!date

## Load Useful functions

In [ ]:
run ptuse_report_useful_functions.py

## Files to be processed

In [ ]:
directory = '/kat/isabella/timing/PTUSE_1_20180417074239_J0437m4715/'

## Set up environment

In [ ]:
import os
import sys
import numpy as np
from matplotlib import cm
from matplotlib import colors
import psrchive as psr
# from coast_guard import cleaners
import time
from astropy.table import Table
import subprocess

In [ ]:
%pylab inline

## Load data

In [ ]:
# load and add archive files
archives = load_archive_data(path=directory, verbose=True)

## Display metadata

In [ ]:
nbin = archives[0].get_nbin()
nchan = archives[0].get_nchan()
npol = archives[0].get_npol()
nsubint = archives[0].get_nsubint()
obs_type = archives[0].get_type()
telescope_name = archives[0].get_telescope()
source_name = archives[0].get_source()
ra = archives[0].get_coordinates().ra()
dec = archives[0].get_coordinates().dec()
centre_frequency = archives[0].get_centre_frequency()
bandwidth = archives[0].get_bandwidth()
dm = archives[0].get_dispersion_measure()
rm = archives[0].get_rotation_measure()
is_dedispersed = archives[0].get_dedispersed()
is_faraday_rotated = archives[0].get_faraday_corrected()
is_pol_calib = archives[0].get_poln_calibrated()
data_units = archives[0].get_scale()
data_state = archives[0].get_state()
obs_duration = archives[0].integration_length()
receiver_name = archives[0].get_receiver_name()
receptor_basis = archives[0].get_basis()
backend_name = archives[0].get_backend_name()
low_freq = archives[0].get_centre_frequency() - archives[0].get_bandwidth() / 2.0
high_freq = archives[0].get_centre_frequency() + archives[0].get_bandwidth() / 2.0
subint_duration = np.rint(archives[0].get_Integration(1).get_duration())

In [ ]:
print_metadata(archive=archives[0])

## Clean archive, display RFI stats and RFI mask

In [ ]:
# Run coast_guard RFI cleaning routines
clean_rfi = archives[0].clone()
clean_archive_surgical(clean_rfi, chan_numpieces=1, subint_numpieces=1, chan_threshold=3, subint_threshold=3)
clean_archive_rcvrstd(clean_rfi)
clean_archive_bandwagon(clean_rfi)

In [ ]:
weights = clean_rfi.get_weights()
plot_mask_channels(weights, low_freq, high_freq, nchan, nsubint, target=source_name)

In [ ]:
print_zero_weight_statistics(weights, nchan, nsubint)

## Plot subintegration stacks (Coherency data) & average profiles (Stokes data)

In [ ]:
clean_profile_data = prepare_profile_data(clean_rfi)
dirty_profile_data = prepare_profile_data(archives[0])

In [ ]:
dirty_stack_data = prepare_subint_data(archives=archives[0])

In [ ]:
clean_stack_data = prepare_subint_data(archives=clean_rfi)

In [ ]:
fig, ax, clean_stack, clean_profile, unclean_stack, unclean_profile = create_subint_profile_figure(fig_title='Sub-intergration stack')
plot_stokes_profiles(clean_profile_data, source_name, ax=clean_profile, nbin=nbin, label='Clean data')
plot_stokes_profiles(dirty_profile_data, source_name, ax=unclean_profile, nbin=nbin, label='Unclean data')
plot_sub_intergration_stack(dirty_stack_data, ax=unclean_stack, title=False, nbin=nbin, nsubint=nsubint, obs_duration=obs_duration)
plot_sub_intergration_stack(clean_stack_data, ax=clean_stack, title=False, nbin=nbin, nsubint=nsubint, obs_duration=obs_duration)
set_phase_label(ax=clean_stack)
set_phase_label(ax=unclean_stack)
set_subint_time_label(ax=clean_stack, intergration=obs_duration)
set_subint_time_label(ax=unclean_stack, intergration=obs_duration)
clean_stack.set_ylabel('Subint number')
unclean_stack.set_ylabel('Subint number')

## Plot phase vs. frequency image of flux (clean Coherency data)

In [ ]:
#coherency_flux_phase_data = prepare_coherency_flux_data(archives[0])
coherency_flux_phase_data = prepare_coherency_flux_data(clean_rfi)

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.015)
fig.text(0.05, 1.0, 'Clean coherency data', fontsize='large')
ax[0, 0].set_ylabel('Frequency (MHz)')
ax[1, 0].set_ylabel('Frequency (MHz)')
ax[1, 0].set_xlabel('Pulse phase (bin)')
ax[1, 1].set_xlabel('Pulse phase (bin)')
plot_phase_freq_flux(coherency_flux_phase_data[0, :, :], ax=ax[0, 0], title='XX', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(coherency_flux_phase_data[1, :, :], ax=ax[0, 1], title='YY', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(coherency_flux_phase_data[2, :, :], ax=ax[1, 0], title='Re(XY)', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(coherency_flux_phase_data[3, :, :], ax=ax[1, 1], title='Im(XY)', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
set_phase_label(ax[1, 0])
set_phase_label(ax[1, 1])
set_channel_label(ax[1, 1], nchan)
set_channel_label(ax[0, 1], nchan)

## Plot phase vs. frequency image of flux (clean Stokes data)

In [ ]:
stokes_flux_phase_data = prepare_sokes_phase_freq(clean_rfi)

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean stokes data', fontsize='large')
ax[0, 0].set_ylabel('Frequency (MHz)')
ax[1, 0].set_ylabel('Frequency (MHz)')
ax[1, 0].set_xlabel('Pulse phase (bin)')
ax[1, 1].set_xlabel('Pulse phase (bin)')
plot_phase_freq_flux(stokes_flux_phase_data[0, :, :], ax=ax[0, 0], title='I', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(stokes_flux_phase_data[1, :, :], ax=ax[0, 1], title='Q', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(stokes_flux_phase_data[2, :, :], ax=ax[1, 0], title='U', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
plot_phase_freq_flux(stokes_flux_phase_data[3, :, :], ax=ax[1, 1], title='V', low_freq=low_freq, high_freq=high_freq, nbin=nbin, nchan=nchan)
set_phase_label(ax[1, 0])
set_phase_label(ax[1, 1])
set_channel_label(ax[1, 1], nchan)
set_channel_label(ax[0, 1], nchan)

## Plot bandpass (unclean and clean Stokes data)

In [ ]:
dirty_mean_bandpass, dirty_variance_bandpass = get_mean_bandpass(archive=archives[0], stokes=False)
dirty_stokes_mean_bandpass, dirty_stokes_variance_bandpass = get_mean_bandpass(archive=archives[0], stokes=True)

In [ ]:
clean_mean_bandpass, clean_variance_bandpass = get_mean_bandpass(archive=clean_rfi, stokes=False)
clean_stokes_mean_bandpass, clean_stokes_variance_bandpass = get_mean_bandpass(archive=clean_rfi, stokes=True)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex='col', figsize=[15, 10], tight_layout='true')
set_bandpass_plot_label(dirty_mean_bandpass, target=source_name, title='Unclean data', ax=ax[0], nchan=nchan, logscale=True)
if np.max(dirty_mean_bandpass[1, :]) > np.max(dirty_mean_bandpass[0, :]):
    ax[0].plot(dirty_mean_bandpass[1, :], label='YY')
    ax[0].plot(dirty_mean_bandpass[0, :], label='XX')
else:
    ax[0].plot(dirty_mean_bandpass[0, :], label='XX')
    ax[0].plot(dirty_mean_bandpass[1, :], label='YY')
ax[0].plot(dirty_stokes_mean_bandpass[0, :], label='Stokes I')    
ax[0].fill_between(x=np.linspace(0, np.shape(dirty_mean_bandpass[0, :]), num=len(dirty_mean_bandpass[0, :]), endpoint=True),
                   y1=dirty_mean_bandpass[1, :].min(), y2=dirty_mean_bandpass[1, :].max(),
                   where=(clean_mean_bandpass[0, :] == 0.0), alpha=0.5, color='grey')
ax[0].set_ylim(dirty_mean_bandpass[1, :].min(), dirty_mean_bandpass[1, :].max())
ax[0].legend()

# Plot the clean bandpass:
set_freq_label(ax[1], low_freq, high_freq, pos='xaxis')
set_bandpass_plot_label(clean_mean_bandpass, source_name, 'Clean data', ax[1], nchan, logscale=True)
clean_mean_bandpass[clean_mean_bandpass == 0.0] = np.nan
if np.max(clean_mean_bandpass[1, :]) > np.max(clean_mean_bandpass[0, :]):
    ax[1].plot(clean_mean_bandpass[1, :], label='YY')
    ax[1].plot(clean_mean_bandpass[0, :], label='XX')
else:
    ax[1].plot(clean_mean_bandpass[0, :], label='XX')
    ax[1].plot(clean_mean_bandpass[1, :], label='YY')
ax[1].set_ylim(np.nanmin(dirty_mean_bandpass[1, :]), np.nanmax(dirty_stokes_mean_bandpass[0, :]))   
clean_stokes_mean_bandpass[clean_stokes_mean_bandpass == 0.0] = np.nan    
ax[1].plot(clean_stokes_mean_bandpass[0, :], label='Stokes I')    
ax[1].legend()

## Plot dynamic baseline spectrum (unclean Coherency data)

In [ ]:
unclean_mean_spectrum, unclean_varience_spectrum = get_baseline_spectrum(archives[0], npol, nchan, nsubint)
clean_mean_spectrum, clean_variance_spectrum = get_baseline_spectrum(clean_rfi, npol, nchan, nsubint, stokes=True)

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Unclean data', fontsize='large')

ax[0, 0].set_ylabel('Channel number')
ax[1, 0].set_ylabel('Channel number')
ax[1, 0].set_xlabel('Subint number')
ax[1, 1].set_xlabel('Subint number')

mean_xx_min = unclean_mean_spectrum[:, 0, :].min()
mean_xx_max = unclean_mean_spectrum[:, 0, :].max()
mean_yy_min = unclean_mean_spectrum[:, 1, :].min()
mean_yy_max = unclean_mean_spectrum[:, 1, :].max()
mean_re_xy_min = unclean_mean_spectrum[:, 2, :].min()
mean_re_xy_max = unclean_mean_spectrum[:, 2, :].max()
mean_im_xy_min = unclean_mean_spectrum[:, 3, :].min()
mean_im_xy_max = unclean_mean_spectrum[:, 3, :].max()

plot_dynamic_spectrum(unclean_mean_spectrum[:, 0, :], ax[0, 0], 'XX', low_freq, high_freq, nbin, nchan, 
                      mean_xx_min, mean_xx_max, clean_data=False)
plot_dynamic_spectrum(unclean_mean_spectrum[:, 1, :], ax[0, 1], 'YY', low_freq, high_freq, nbin, nchan,
                      mean_yy_min, mean_yy_max, clean_data=False)
plot_dynamic_spectrum(unclean_mean_spectrum[:, 2, :], ax[1, 0], 'Re(XY)', low_freq, high_freq, nbin, nchan,
                      mean_re_xy_min, mean_re_xy_max, clean_data=False)
plot_dynamic_spectrum(unclean_mean_spectrum[:, 3, :], ax[1, 1], 'Im(XY)', low_freq, high_freq, nbin, nchan, 
                      mean_im_xy_min, mean_im_xy_max, clean_data=False)




## Plot dynamic baseline spectrum (clean Stokes data)

In [ ]:
fig, ax = plt.subplots(2, 2, sharex='col', sharey='row', figsize=[15, 10], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')

variance_xx_min = clean_variance_spectrum[:, 0, :].min()
variance_xx_max = clean_variance_spectrum[:, 0, :].max()
variance_yy_min = clean_variance_spectrum[:, 1, :].min()
variance_yy_max = clean_variance_spectrum[:, 1, :].max()
variance_re_xy_min = clean_variance_spectrum[:, 2, :].min()
variance_re_xy_max = clean_variance_spectrum[:, 2, :].max()
variance_im_xy_min = clean_variance_spectrum[:, 3, :].min()
variance_im_xy_max = clean_variance_spectrum[:, 3, :].max()

plot_dynamic_spectrum(clean_variance_spectrum[:, 0, :], ax[0, 0], 'I', low_freq, high_freq, nbin, nchan, 
                      variance_xx_min, variance_xx_max, clean_data=True)
plot_dynamic_spectrum(clean_variance_spectrum[:, 1, :], ax[0, 1], 'Q', low_freq, high_freq, nbin, nchan,
                      variance_yy_min, variance_yy_max, clean_data=True)
plot_dynamic_spectrum(clean_variance_spectrum[:, 2, :], ax[1, 0], 'U', low_freq, high_freq, nbin, nchan,
                      variance_re_xy_min, variance_re_xy_max, clean_data=True)
plot_dynamic_spectrum(clean_variance_spectrum[:, 3, :], ax[1, 1], 'V', low_freq, high_freq, nbin, nchan, 
                      variance_im_xy_min, variance_im_xy_max, clean_data=True)

## Plot signal-to-noise ratio vs. subintegration (clean Coherency data)

In [ ]:
snr_data = get_subint_snr(clean_rfi, nsubint)

In [ ]:
fig, ax1 = plt.subplots(1, 1, sharex='col', sharey='row', figsize=[15, 5], tight_layout='true')
fig.suptitle(source_name, fontsize='large', y=1.01)
fig.text(0.05, 1.0, 'Clean data', fontsize='large')
ax1.set_xlabel('Subint number')
ax1.set_ylabel('Signal-to-noise ratio')
ax1.set_xlim(0, nsubint)
ax1_secondary = ax1.twiny()
ax1_secondary.set_frame_on(True)
ax1_secondary.patch.set_visible(False)
ax1_secondary.xaxis.set_ticks_position('bottom')
ax1_secondary.set_xlabel('Time (seconds)')
ax1_secondary.xaxis.set_label_position('bottom')
ax1_secondary.spines['bottom'].set_position(('outward', 50))
ax1_secondary.set_xlim(0, obs_duration)
ax1.plot(snr_data)

In [ ]:
print 'Report Ended:'
!date